In [ ]:
import cv2
import tkinter as tk

class ImageLoaderApp:
    def __init__(self, master):
        self.master = master
        self.master.title("Image Loader")

        self.reference_image_path = "/home/sheharyaar/Downloads/dataset/reference1Km.tif"
        self.aerial_image_paths = ["/home/sheharyaar/Downloads/dataset/aerial1.png", "/home/sheharyaar/Downloads/dataset/aerial2.png", "/home/sheharyaar/Downloads/dataset/aerial3.png", "/home/sheharyaar/Downloads/dataset/aerial4.png", "/home/sheharyaar/Downloads/dataset/aerial5.png", "/home/sheharyaar/Downloads/dataset/aerial6.png", "/home/sheharyaar/Downloads/dataset/aerial7.png", "/home/sheharyaar/Downloads/dataset/aerial8.png", "/home/sheharyaar/Downloads/dataset/aerial9.png", "/home/sheharyaar/Downloads/dataset/aerial10.png"]

        self.reference_button = tk.Button(master, text="Load Reference Image", command=self.load_reference_image, width=20)
        self.reference_button.grid(row=0, column=0, padx=10, pady=10)

        self.aerial_button = tk.Button(master, text="Load Aerial Images", command=self.load_aerial_images, width=20)
        self.aerial_button.grid(row=1, column=0, padx=10, pady=10)

        self.quit_button = tk.Button(master, text="Quit", command=master.quit, width=20)
        self.quit_button.grid(row=2, column=0, padx=10, pady=10)

    def load_reference_image(self):
        self.reference_image = cv2.imread(self.reference_image_path)
        self.show_image(self.reference_image, "Reference Image")

    def load_aerial_images(self):
        for i, image_path in enumerate(self.aerial_image_paths, 1):
            aerial_image = cv2.imread(image_path)
            self.show_image(aerial_image, f"Aerial Image {i}")

    def show_image(self, image, title):
        cv2.imshow(title, image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def main():
    root = tk.Tk()
    app = ImageLoaderApp(root)
    root.mainloop()

if __name__ == "__main__":
    main()


QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to target thread (0x5555e0cb0900)

QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to target thread (0x5555e0cb0900)

QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to target thread (0x5555e0cb0900)

QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to target thread (0x5555e0cb0900)

QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to target thread (0x5555e0cb0900)

QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to target thread (0x5555e0cb0900)

QObject::moveToThread: Current thread (0x5555e0cb0900) is not the object's thread (0x5555e1d52100).
Cannot move to tar

In [ ]:
import cv2
import numpy as np

# Step 1: Load reference and aerial images
def load_images(reference_path, aerial_path):
    reference_image = cv2.imread(reference_path)
    aerial_image = cv2.imread(aerial_path)
    return reference_image, aerial_image

# Step 2: Function to handle mouse click events
def get_mouse_click(event, x, y, flags, param):
    global clicked_point
    if event == cv2.EVENT_LBUTTONDOWN:
        clicked_point = (x, y)

# Step 3: Compute keypoints and descriptors
def compute_keypoints_and_descriptors(image, detector):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    keypoints, descriptors = detector.detectAndCompute(gray, None)
    return keypoints, descriptors

# Step 4: Compute Homography matrix
def compute_homography(reference_keypoints, reference_descriptors, aerial_keypoints, aerial_descriptors):
    matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_BRUTEFORCE)
    matches = matcher.match(aerial_descriptors, reference_descriptors)
    matches = sorted(matches, key=lambda x: x.distance)
    good_matches = matches[:50]  # Adjust this parameter based on your needs
    src_pts = np.float32([aerial_keypoints[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([reference_keypoints[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H

# Step 5: Project clicked pixel location
def project_pixel_location(H, clicked_point):
    x, y = clicked_point
    points = np.array([[x, y]], dtype='float32')
    projected_points = cv2.perspectiveTransform(points.reshape(-1, 1, 2), H)
    return projected_points[0][0]

# Step 6: Convert projected pixel location to latitude and longitude
def pixel_to_lat_long(projected_point, reference_lat_long, pixel_resolution):
    # You need to implement this part based on your specific use case
    # Consider the pixel resolution and the reference latitude and longitude
    return lat_long

# Step 7: Display coordinates to the user
def display_coordinates(image, coordinates):
    cv2.putText(image, f"Lat: {coordinates[0]}, Long: {coordinates[1]}", (20, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return image

# Main function
def main():
    reference_image, aerial_image = load_images("/home/sheharyaar/Downloads/dataset/reference1Km.tif", "/home/sheharyaar/Downloads/dataset/aerial1.png")
    detector = cv2.SIFT_create()  # You can use other detectors as well
    aerial_keypoints, aerial_descriptors = compute_keypoints_and_descriptors(aerial_image, detector)
    reference_keypoints, reference_descriptors = compute_keypoints_and_descriptors(reference_image, detector)

    # Step 2: Handle mouse click events
    global clicked_point
    clicked_point = None
    cv2.namedWindow("Aerial Image")
    cv2.setMouseCallback("Aerial Image", get_mouse_click)

    while True:
        cv2.imshow("Aerial Image", aerial_image)
        if clicked_point:
            H = compute_homography(reference_keypoints, reference_descriptors, aerial_keypoints, aerial_descriptors)
            projected_point = project_pixel_location(H, clicked_point)
            # Convert projected pixel location to latitude and longitude
            # lat_long = pixel_to_lat_long(projected_point, reference_lat_long, pixel_resolution)
            # Display coordinates to the user
            # aerial_image = display_coordinates(aerial_image, lat_long)
            print("Projected Pixel Location:", projected_point)
            clicked_point = None

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to tar

Projected Pixel Location: [2538.168   998.1195]
Projected Pixel Location: [2540.1936   991.35657]
Projected Pixel Location: [2540.1936   991.35657]


QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to tar

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to tar

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to tar

Projected Pixel Location: [3070.3547   -41.24836]
Projected Pixel Location: [2394.1606 1242.6604]
Projected Pixel Location: [2526.0688  983.6949]
Projected Pixel Location: [2548.4558  985.515 ]


QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to target thread (0x7fa9dc000db0)

QObject::moveToThread: Current thread (0x7fa9dc000db0) is not the object's thread (0x5625692c7710).
Cannot move to tar